## Day 20

https://adventofcode.com/2020/day/20

In [1]:
import collections
import copy
import enum
import itertools
import math
import typing

In [2]:
import aocd

In [3]:
test_data = """
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
"""

In [4]:
Lines = typing.List[str]

In [5]:
def parse_tile_items(line: str) -> typing.Tuple[int, Lines]:
    first, *rest = line.split('\n')
    assert 'Tile ' in first
    parts = first.split(' ')
    tile_id = int(parts[1][:-1])
    assert len(rest) == len(rest[0]) == len(rest[-1])  # square
    return tile_id, rest

In [6]:
def parse_data(data: str) -> typing.Dict[int, Lines]:
    data = data[1:] if data[0] == '\n' else data
    lines = [line.rstrip('\n') for line in data.split('\n\n')]
    return dict(parse_tile_items(line) for line in lines)

In [7]:
def pretty_print(lines: Lines) -> None:
    print('\n' + '\n'.join(line for line in lines) + '\n')

In [8]:
# tiles = parse_data(test_data)
tiles = parse_data(aocd.get_data(day=20, year=2020))
len(tiles)

144

In [9]:
def transpose(lines: Lines) -> Lines:
    """Flips across main diagonal."""
    return list(''.join(line) for line in zip(*lines))

In [10]:
def test_transpose(tile) -> bool:
    return transpose(transpose(tile)) == tile

In [11]:
assert all(test_transpose(tiles[tile]) for tile in tiles)

In [12]:
def flip(lines: Lines) -> Lines:
    """Flips horizontally."""
    return [''.join(reversed(line)) for line in lines]

In [13]:
def test_flip(tile) -> bool:
    return flip(flip(tile)) == tile

In [14]:
assert all(test_flip(tiles[tile]) for tile in tiles)

In [15]:
def rotate(lines: Lines) -> Lines:
    """Rotates 90 degrees counter-clockwise."""
    return transpose(flip(lines))

In [16]:
def test_rotate(tile) -> bool:
    return rotate(rotate(rotate(rotate(tile)))) == tile

In [17]:
assert all(test_rotate(tiles[tile]) for tile in tiles)

In [18]:
class Side(enum.Enum):
    TOP = 0
    RIGHT = 1
    BOTTOM = 2
    LEFT = 3
    
    def complement(self):
        return Side((self.value + 2) % 4)

In [19]:
class Orientation(enum.Enum):
    UNCHANGED = 0
    ROTATE_90 = 1
    ROTATE_180 = 2
    ROTATE_270 = 3
    FLIP_HORIZONTAL = 4
    FLIP_ROTATE_90 = 5
    FLIP_ROTATE_180 = 6
    FLIP_ROTATE_270 = 7

    @property
    def flipped(self) -> bool:
        return self.value > 3
    
    @property
    def rotations(self) -> int:
        return self.value % 4
    
    def apply(self, lines: Lines) -> Lines:
        lines = copy.deepcopy(lines)
        if self.flipped:
            lines = flip(lines)
        for _ in range(self.rotations):
            lines = rotate(lines)
        return lines
    
    @staticmethod
    def generate_all(lines: Lines) -> typing.List[Lines]:
        group = [copy.deepcopy(lines)]
        for _ in range(3):
            group.append(rotate(group[-1]))
        group.append(flip(lines))
        for _ in range(3):
            group.append(rotate(group[-1]))
        return group

In [20]:
class Borders(typing.NamedTuple):
    top: str
    right: str
    bottom: str
    left: str

    @classmethod
    def from_lines(cls, lines: Lines):
        top = lines[0]
        bottom = lines[-1]
        left = ''.join(line[0] for line in lines)
        right = ''.join(line[-1] for line in lines)
        return cls(top=top, right=right, bottom=bottom, left=left)
    
    def get(self, side: Side) -> str:
        return self[side.value]
    
    def pretty_print(self):
        n = len(self.top)
        lines = [self.top]
        for i in range(1, n-1):
            lines.append(self.left[i] + ' '*(n-2) + self.right[i])
        lines.append(self.bottom)
        print('\n'.join(lines))

In [21]:
class BorderGroup:
    def __init__(self, group: typing.List[Lines]):
        self.borders = {
            Orientation(i): Borders.from_lines(lines)
            for i, lines in enumerate(group)
        }
        self.unique_values = set([
            border
            for value in self.borders.values()
            for border in value
        ])
    
    def __repr__(self):
        return repr(self.borders)
    
    def __getitem__(self, orientation: Orientation):
        return self.borders[orientation]
    
    def get(self, *, side: Side, orientation: Orientation) -> str:
        return self.borders[orientation].get(side)
    
    def overlap(self, other) -> int:
        return len(self.unique_values & other.unique_values)
    
    def matches(self, other) -> bool:
        for border in self.unique_values:
            for other_border in other.unique_values:
                if border == other_border:
                    return True
        return False

In [22]:
class Alignment(typing.NamedTuple):
    side: Side
    orientation: Orientation
    neighbor_orientation: Orientation

In [23]:
class Tile(typing.NamedTuple):
    raw: Lines
    borders: BorderGroup
    
    @classmethod
    def from_lines(cls, lines: Lines):
        group = Orientation.generate_all(lines)
        return cls(raw=lines, borders=BorderGroup(group))

    def get_image(self, orientation: Orientation) -> Lines:
        image = orientation.apply(self.raw)
        return [line[1:-1] for line in image[1:-1]]    

    def overlap(self, other) -> int:
        return self.borders.overlap(other.borders)
    
    def matches(self, other) -> bool:
        return self.borders.matches(other.borders)

    def find_alignments(
        self,
        other,
        *,
        side: Side,
    ) -> typing.List[Alignment]:
        other_side = side.complement()
        alignments = []
        orientations = itertools.product(Orientation, repeat=2)
        for orientation, other_orientation in orientations:
            border = self.borders.get(
                side=side,
                orientation=orientation,
            )
            other_border = other.borders.get(
                side=other_side,
                orientation=other_orientation,
            )
            if border == other_border:
                alignments.append(Alignment(
                    side=side,
                    orientation=orientation,
                    neighbor_orientation=other_orientation,
                ))
        return alignments

In [24]:
def preprocess_data(data) -> typing.Dict[int, Tile]:
    tiles = parse_data(data)
    return {
        tile_id: Tile.from_lines(raw)
        for tile_id, raw in tiles.items()
    }

In [25]:
# tiles = preprocess_data(test_data)
tiles = preprocess_data(aocd.get_data(day=20, year=2020))
len(tiles)

144

In [26]:
list(tiles.keys())

[1217,
 2357,
 1399,
 3733,
 2503,
 1511,
 3583,
 1637,
 1453,
 1153,
 3701,
 2179,
 1999,
 2333,
 1733,
 3217,
 1471,
 2089,
 2909,
 3433,
 3049,
 2143,
 3299,
 2803,
 2311,
 1123,
 1777,
 1321,
 2287,
 3727,
 1607,
 1327,
 3889,
 1447,
 3767,
 2957,
 3911,
 2683,
 2267,
 1709,
 2347,
 1051,
 2903,
 1619,
 3203,
 2269,
 2027,
 1973,
 3559,
 3319,
 3631,
 2377,
 3331,
 2473,
 1039,
 2111,
 2039,
 3391,
 1913,
 2693,
 1033,
 2273,
 1049,
 1787,
 2689,
 1933,
 1823,
 1439,
 1657,
 3517,
 3943,
 2801,
 2411,
 3907,
 2819,
 3307,
 3313,
 3037,
 2053,
 3769,
 1759,
 1229,
 3821,
 1319,
 3167,
 3407,
 1231,
 3623,
 3541,
 2939,
 1847,
 2887,
 3697,
 2081,
 2383,
 2557,
 3637,
 3989,
 2221,
 1523,
 1583,
 3919,
 2381,
 3923,
 2953,
 2441,
 3023,
 2659,
 1069,
 1181,
 1663,
 3779,
 1013,
 1747,
 3931,
 2593,
 3677,
 2633,
 2129,
 2029,
 1873,
 1567,
 3571,
 2671,
 3229,
 3083,
 2239,
 3581,
 1061,
 3967,
 2749,
 1811,
 2477,
 1949,
 2099,
 2767,
 3251,
 3833,
 3089,
 2861,
 1489,
 1801,
 2677,

In [27]:
class CornerType(enum.Enum):
    TOP_LEFT = (Side.RIGHT, Side.BOTTOM)
    TOP_RIGHT = (Side.LEFT, Side.BOTTOM)
    BOTTOM_RIGHT = (Side.LEFT, Side.TOP)
    BOTTOM_LEFT = (Side.RIGHT, Side.TOP)

In [28]:
class MatrixTile(typing.NamedTuple):
    tile_id: int
    orientation: Orientation
    image: Lines

In [29]:
class AdjacencyMatrix(collections.abc.Mapping):
    def __init__(self, tiles: typing.Dict[int, Tile]):
        n = len(tiles)
        assert n == math.isqrt(n) ** 2  # grid must be square
        self.tiles = dict(tiles)
        self._matrix = None
        self._image = None
        self.adjacency = collections.defaultdict(list)
        for i, tile1 in tiles.items():
            others = set(tiles.keys())
            others.remove(i)
            for j in others:
                if tile1.matches(tiles[j]):
                    self.adjacency[i].append(j)
    
    def __repr__(self):
        return repr(self._adjacency)
    
    def __getitem__(self, tile_id):
        return self.adjacency[tile_id]
    
    def __len__(self):
        return len(self.adjacency)
    
    def __iter__(self):
        return iter(self.adjacency)
    
    def count_adjacent(self, tile_id: int) -> int:
        return len(self[tile_id])
    
    def is_corner(self, tile_id: int) -> bool:
        return self.count_adjacent(tile_id) == 2

    @property
    def overlaps(self) -> typing.DefaultDict:
        overlaps = collections.defaultdict(list)
        for i in tiles:
            for j in tiles:
                if j == i:
                    continue
                overlap = self.tiles[i].overlap(tiles[j])
                if overlap > 0:
                    overlaps[i].append((j, overlap))
        return overlaps
    
    @property
    def corners(self) -> typing.List[int]:
        return [tile for tile in self if self.is_corner(tile)]
    
    def _corner_alignment(
            self,
            *,
            corner_number: int,
            corner_type: CornerType, 
    ) -> typing.Dict:
        corner_id = self.corners[corner_number]
        corner_tile = self.tiles[corner_id]
        neighbor_ids = self[corner_id]
        alignments = [
            (n, a)
            for n in neighbor_ids
            for side in corner_type.value
            for a in corner_tile.find_alignments(
                self.tiles[n],
                side=side,
            )
        ]
        candidates = collections.defaultdict(list)
        for n, a in alignments:
            key = (corner_id, a.orientation)
            value = (a.side, (n, a.neighbor_orientation))
            candidates[key].append(value)
        return {
            key: value
            for key, value in candidates.items()
            if len(value) == 2
        }
    
    def adjacent_alignment(
            self,
            tile: MatrixTile,
            *,
            side: Side,
            exclude_ids: typing.Optional[typing.Set] = None,
    ) -> MatrixTile:
        if exclude_ids is None:
            exclude_ids = set()
        neighbor_ids = self[tile.tile_id]
        alignments = [
            (n, a)
            for n in neighbor_ids if n not in exclude_ids
            for a in self.tiles[tile.tile_id].find_alignments(
                self.tiles[n],
                side=side,
            ) if a.orientation == tile.orientation
        ]
        assert len(alignments) == 1
        tile = alignments[0]
        tile_id = tile[0]
        orientation = tile[1].neighbor_orientation
        return MatrixTile(
            tile_id=tile_id,
            orientation=orientation,
            image=self.tiles[tile_id].get_image(orientation),
        )
    
    def matrix_top_left(self):
        n = math.isqrt(len(self.tiles))
        matrix = [[None for _ in range(n)] for _ in range(n)]
        candidates = self._corner_alignment(
            corner_number=0,
            corner_type=CornerType.TOP_LEFT,
        )
        top_left = list(candidates.items())[0]
        corner_id, corner_orientation = top_left[0]
        neighbors = dict(top_left[1])
        right_id, right_orientation = neighbors[Side.RIGHT]
        bottom_id, bottom_orientation = neighbors[Side.BOTTOM]
        matrix[0][0] = MatrixTile(
            tile_id=corner_id,
            orientation=corner_orientation,
            image=self.tiles[corner_id].get_image(corner_orientation),
        )
        matrix[0][1] = MatrixTile(
            tile_id=right_id,
            orientation=right_orientation,
            image=self.tiles[right_id].get_image(right_orientation),
        )
        matrix[1][0] = MatrixTile(
            tile_id=bottom_id,
            orientation=bottom_orientation,
            image=self.tiles[bottom_id].get_image(bottom_orientation),
        )
        return matrix
    
    @property
    def matrix(self) -> typing.List[typing.List[MatrixTile]]:
        if self._matrix is not None:
            return self._matrix
        matrix = self.matrix_top_left()
        n = len(matrix)
        exclude_ids = set([
            tile.tile_id
            for row in matrix
            for tile in row if tile is not None
        ])
        for j in range(2, n):
            matrix[0][j] = self.adjacent_alignment(
                matrix[0][j-1],
                side=Side.RIGHT,
            )
        for j in range(1, n):
            matrix[1][j] = self.adjacent_alignment(
                matrix[1][j-1],
                side=Side.RIGHT,
            )         
        for i in range(2, n):
            matrix[i][0] = self.adjacent_alignment(
                matrix[i-1][0],
                side=Side.BOTTOM,
            )
            for j in range(1, n):
                matrix[i][j] = self.adjacent_alignment(
                    matrix[i][j-1],
                    side=Side.RIGHT,
                )                         
        self._matrix = matrix
        return self._matrix
    
    @property
    def image(self) -> Lines:
        if self._image is not None:
            return self._image
        image = []
        for row in self.matrix:
            tiles = iter(row)
            tile = next(tiles)
            lines = [line for line in tile.image]
            for tile in tiles:
                for i, line in enumerate(tile.image):
                    lines[i] = f'{lines[i]}{line}'
            image.extend(lines)
        self._image = image
        return self._image

In [30]:
adj = AdjacencyMatrix(tiles)
len(adj)

144

### Solution to Part 1

In [31]:
math.prod(adj.corners)

15006909892229

### Solution to Part 2

In [32]:
pretty_print(adj.image)


.......###.#......#....#...#.....#...#.......#..##.#.#...#...##.#.#.....###..##......#......#.#.
...#....#...#....#.......#.........#..#..#.....#.##.....#...##.#......#...#..#....###...#.#.....
...#.......###...#.#...##.....#.#....#...#.#..#....#................#...#.#....#....##..#..#.#..
#..#.##....#..........#.........#...#.....#.......#..####..#.##..#.###...#.#...........#...##...
#.#.##.................##...#.#....#..#....##......#..#..#..#..#.##..#.....................#....
.#....###..#........#...##....#......#....##...........##.#.#..#..##...#...#.##.##..#..#.....###
.#.....##....#.#..#.##..#....#...##...#.#...#.....#..#..#.....#..#...............#..#.#..#.#..#.
####...#.....#..#..#.........##...#...#..#.....#....##.#.#.#..##.....##....#..........#.##....#.
#...#..##.....#.#..####....##....###...#...##..##....#............#....#.......##.....#......#.#
#..##.#...#......#.##.###.#..#..#.##...#...##...............##...#.#....#..#....#.#............#
..##..#..#.##...#...........#

In [33]:
SEA_MONSTER = """
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
"""

In [34]:
class SeaMonster:
    def __init__(self, pattern: Lines):
        self.pattern = pattern
        self.rows = len(pattern)
        self.cols = len(pattern[0])
        assert all(len(row) == self.cols for row in pattern)
        self.pounds = ''.join(adj.image).count('#')
        
    @classmethod
    def from_text(cls, text: str):
        return cls(text.split('\n')[1:-1])
    
    def __repr__(self):
        return f'{self.__class__.__name__}({str(self.pattern)})'
    
    def pretty_print(self) -> None:
        return pretty_print(self.pattern)
    
    def magnifier(self, image, *, row: int, col: int) -> None:
        lines = [
            row[col:(col + self.cols + 1)]
            for row in image[row:(row + self.rows + 1)]
        ]
        pretty_print(lines)
    
    def mark_pounds(self, *, row: int, col: int) -> typing.Set:
        return set(
            (row + j, col + i)
            for j in range(self.rows)
            for i in range(self.cols)
            if self.pattern[j][i] == '#'
        )
        
    def check_match(self, image: Lines, *, row: int, col: int) -> bool:
        return all(
            self.pattern[j][i] == ' ' or image[row+j][col+i] == '#'
            for j in range(self.rows)
            for i in range(self.cols)
        )

    def count_matches(self, image: Lines) -> int:
        sea_monster_pounds = set()
        rows = len(image)
        cols = len(image[0])
        found = 0
        for row in range(rows - self.rows + 1):
            for col in range(cols - self.cols + 1):
                if self.check_match(image, row=row, col=col):
                    found += 1
                    sea_monster_pounds |= self.mark_pounds(row=row, col=col)
        water_pounds = [
            '#' for j in range(rows) for i in range(cols)
            if (j, i) not in sea_monster_pounds and image[j][i] == '#'
        ]
        return found, len(water_pounds)
    
    def search_all(self, image: Lines) -> int:
        images = {
            Orientation(i): transformed
            for i, transformed in enumerate(Orientation.generate_all(image))
        }
        for orientation, transformed in images.items():
            found, water_pounds = self.count_matches(transformed)
            if found > 0:
                return orientation, found, water_pounds

In [35]:
sea_monster = SeaMonster.from_text(SEA_MONSTER)
sea_monster.pretty_print()


                  # 
#    ##    ##    ###
 #  #  #  #  #  #   



In [36]:
sea_monster.search_all(adj.image)

(<Orientation.UNCHANGED: 0>, 27, 2190)